# Buffer Overflow Protections
- modern kernel/OS and compilers provide various protections from buffer overflow attacks
- Reference: https://ocw.cs.pub.ro/courses/cns/labs/lab-06

## 1. ASLR - Address Space Layout Randomization
- Kernel specific protection
- loads the stack, data and program in random memory locations making it difficult (not impossible) to guess their addresses

- the following segments are possible to randomize on x86:
    - **stack** is easily randomizable, as all stack addresses are relative to `esp` or `ebp`
    - **Data segment** may be randomized, if e.g., the data segment is set to a random value
    - **Code** can only be randomized by comiling the program as Position Independent Code/Position Independent Executable
       - default for shared libraries, but otherwise executable code is usually placed at fixed addresses

### ASLR values
- Linux allows 3 options:
    - 0: disabled
    - 1: randomize stack, vdso, libraries (except data? not sure...)
    - 2: enabled (including data segment - default value)
    
### checking ASLR
- compile and run demos/aslr.cpp program to check ASLR
- run gdb-peda 

    ```bash
    aslr # run aslr command
    ASLR is OFF # output
    ```

In [4]:
! sudo -S cat /proc/sys/kernel/randomize_va_space

2


In [2]:
! echo kali | sudo -S bash -c 'echo 2 | sudo -S tee /proc/sys/kernel/randomize_va_space'

2


In [5]:
! cat ../demos/aslr/aslr.cpp

// aslr.cpp - program to check address space layout randomization
// compile and run the program multiple times
// if you see same value for $esp, ASLR is disabled
// $ g++ -g -m 32 -o aslr.exe aslr.cpp
#include <stdio.h>
#include <iostream>
using namespace std;

int global_var = 100; // data segment
char global_name[20]; // bss segment

int main(void) {
    char buffer[] = "Hello World!";
    register int esp asm("esp");
    register int ebp asm("ebp");
    cout << "Registers info:\n";
    cout << "===============\n";
    printf("$esp = %p\n", esp);
    printf("$ebp = %p\n", ebp);
    cout << "\nData and BSS segement\n";
    cout << "======================\n";
    printf("global_var is at: %p\n", &global_var);
    printf("global_name is at: %p\n", global_name);
    cout << "\nStack variable:\n";
    cout << "===============\n";
    printf("buffer is at: %p\n", buffer);
    cout << "\nCode segment:\n";
    cout << "=============\n";
    printf("main is at %p\n", main);
    cout << "\nS

In [6]:
%%bash
input="../demos/aslr/aslr.cpp"
output="aslr.exe"
g++ -m32 $input -o $output

In file included from ../demos/aslr/aslr.cpp:5:
/usr/include/stdio.h:27:10: fatal error: bits/libc-header-start.h: No such file or directory
   27 | #include <bits/libc-header-start.h>
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~
compilation terminated.


CalledProcessError: Command 'b'input="../demos/aslr/aslr.cpp"\noutput="aslr.exe"\ng++ -m32 $input -o $output\n'' returned non-zero exit status 1.

In [6]:
! ./aslr.exe | tee aslr2-1.txt

Registers info:
$esp = 0xffc3a300
$ebp = 0xffc3a318

Data and BSS segement
global_var is at: 0x5657a018
global_name is at: 0x5657a020

Stack variable:
buffer is at: 0xffc3a303

Code segment:
main is at 0x565771ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [7]:
! ./aslr.exe | tee aslr2-2.txt

Registers info:
$esp = 0xffdc6ab0
$ebp = 0xffdc6ac8

Data and BSS segement
global_var is at: 0x56657018
global_name is at: 0x56657020

Stack variable:
buffer is at: 0xffdc6ab3

Code segment:
main is at 0x566541ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [8]:
# run the diff of aslr2-1.txt and aslr2-2.txt
! diff aslr2-1.txt aslr2-2.txt

3,4c3,4
< $esp = 0xffc3a300
< $ebp = 0xffc3a318
---
> $esp = 0xffdc6ab0
> $ebp = 0xffdc6ac8
8,9c8,9
< global_var is at: 0x5657a018
< global_name is at: 0x5657a020
---
> global_var is at: 0x56657018
> global_name is at: 0x56657020
13c13
< buffer is at: 0xffc3a303
---
> buffer is at: 0xffdc6ab3
17c17
< main is at 0x565771ad
---
> main is at 0x566541ad


In [9]:
# ldd invokes the standard dynamic linker with LD_TRACE_LOADED OBJECTS
# causes the dynamic linker to inspect the program's dynamic dependencies and the objects
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7f83000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e5a000)
	/lib/ld-linux.so.2 (0xf7f85000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e33000)


In [13]:
# see the library loaded in different addresses
# run it few times to see the random addresses...
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc3000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9a000)
	/lib/ld-linux.so.2 (0xf7fc5000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e73000)


### Set ASLR to option 1
- randomize stack, vdso, libraries (except data?)
- actually, global data and bss segments are also randomized!

In [14]:
! echo kali | sudo -S bash -c 'echo 1 | sudo -S tee /proc/sys/kernel/randomize_va_space'

[sudo] password for kali: 1


In [15]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 1



In [16]:
! ./aslr.exe | tee aslr1-1.txt

Registers info:
$esp = 0xff86e510
$ebp = 0xff86e528

Data and BSS segement
global_var is at: 0x5658a018
global_name is at: 0x5658a020

Stack variable:
buffer is at: 0xff86e513

Code segment:
main is at 0x565871ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [17]:
! ./aslr.exe | tee aslr1-2.txt

Registers info:
$esp = 0xff80c2a0
$ebp = 0xff80c2b8

Data and BSS segement
global_var is at: 0x5656e018
global_name is at: 0x5656e020

Stack variable:
buffer is at: 0xff80c2a3

Code segment:
main is at 0x5656b1ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [18]:
! diff aslr1-1.txt aslr1-2.txt

3,4c3,4
< $esp = 0xff86e510
< $ebp = 0xff86e528
---
> $esp = 0xff80c2a0
> $ebp = 0xff80c2b8
8,9c8,9
< global_var is at: 0x5658a018
< global_name is at: 0x5658a020
---
> global_var is at: 0x5656e018
> global_name is at: 0x5656e020
13c13
< buffer is at: 0xff86e513
---
> buffer is at: 0xff80c2a3
17c17
< main is at 0x565871ad
---
> main is at 0x5656b1ad


In [20]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7f8a000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e61000)
	/lib/ld-linux.so.2 (0xf7f8c000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e3a000)


In [21]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fbb000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e92000)
	/lib/ld-linux.so.2 (0xf7fbd000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e6b000)


### Set ASLR to option 0
- disable ASLR

In [22]:
! echo kali | sudo -S bash -c 'echo 0 | sudo tee /proc/sys/kernel/randomize_va_space'

[sudo] password for kali: 0


In [23]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

[sudo] password for kali: 0


In [24]:
! ./aslr.exe | tee aslr0-1.txt

Registers info:
$esp = 0xffffbdd0
$ebp = 0xffffbde8

Data and BSS segement
global_var is at: 0x56559018
global_name is at: 0x56559020

Stack variable:
buffer is at: 0xffffbdd3

Code segment:
main is at 0x565561ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [25]:
! ./aslr.exe | tee aslr0-2.txt

Registers info:
$esp = 0xffffbdd0
$ebp = 0xffffbde8

Data and BSS segement
global_var is at: 0x56559018
global_name is at: 0x56559020

Stack variable:
buffer is at: 0xffffbdd3

Code segment:
main is at 0x565561ad

Shared library code
printf is at 0xf7853df0
cout is at 0xf7e23c40


In [26]:
! diff aslr0-1.txt aslr0-2.txt
# no difference!

In [27]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


In [28]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


### ldd command

- print shared object dependencies
- https://man7.org/linux/man-pages/man1/ldd.1.html
- don't run ldd on untrusted ELF
- use objectdump instead

```bass
 $ objdump -p /path/to/program | grep NEEDED
 ```

In [29]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


In [30]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


In [31]:
# detailed but not as good results as ldd
! objdump -p ./aslr.exe


./aslr.exe:     file format elf32-i386

Program Header:
    PHDR off    0x00000034 vaddr 0x00000034 paddr 0x00000034 align 2**2
         filesz 0x00000160 memsz 0x00000160 flags r--
  INTERP off    0x00000194 vaddr 0x00000194 paddr 0x00000194 align 2**0
         filesz 0x00000013 memsz 0x00000013 flags r--
    LOAD off    0x00000000 vaddr 0x00000000 paddr 0x00000000 align 2**12
         filesz 0x00000518 memsz 0x00000518 flags r--
    LOAD off    0x00001000 vaddr 0x00001000 paddr 0x00001000 align 2**12
         filesz 0x00000440 memsz 0x00000440 flags r-x
    LOAD off    0x00002000 vaddr 0x00002000 paddr 0x00002000 align 2**12
         filesz 0x00000290 memsz 0x00000290 flags r--
    LOAD off    0x00002ed0 vaddr 0x00003ed0 paddr 0x00003ed0 align 2**12
         filesz 0x0000014c memsz 0x00000168 flags rw-
 DYNAMIC off    0x00002edc vaddr 0x00003edc paddr 0x00003edc align 2**2
         filesz 0x000000f8 memsz 0x000000f8 flags rw-
    NOTE off    0x000001a8 vaddr 0x000001a8 paddr 0x00000

#### Allow ptrace processes
**ptrace** - allows parent process to observe and control the execution of another process
- used primarily to implement breakpoint debugging and system call tracing

In [32]:
# check the ptrace value
! echo kali | sudo -S sysctl kernel.yama.ptrace_scope

[sudo] password for kali: kernel.yama.ptrace_scope = 0


In [33]:
# set the ptrace value
! echo kali | sudo -S sysctl -w kernel.yama.ptrace_scope=0

[sudo] password for kali: kernel.yama.ptrace_scope = 0


### 2. PIE
- Position Independent Executable
- randomizes Code segment base address
- randomizes GOT (Global Offset Table for global data-variables) and PLT (Procedure Linkage Table)
    - PLT addresses are fixed during  linking to load the libc.so library codes/functions
    - shared libraries are position independent code (PIC); they don't know where they are being loaded
- disable PIE in gcc/g++ using **-no-pie** flag

### 3. Executable Stack Protection (NX) also called (DEP)
- also called DEP - Data Execution Prevention
- in newer g++, programs must be compiled by disabling stack protector to execute code injected into stack
- check executable stack:
`$ readelf -l <filename>`
- note the line: 
GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10
RW 
- RW - Read and Write but not Executable
- RWE - Read, Write, and Executable

In [34]:
%%bash
in=./demos/hello.cpp
out=hello
g++ -g -m32 -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10


#### enable RWX/disable DEP
- compile the program using -z execstack switch of gcc/g++

In [35]:
%%bash
in=./demos/hello.cpp
out=hello
g++ -g -m32 -z execstack -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RWE 0x10


### 4. Stack Canaries
- "canary in a coal mine" to detect carbon monoxide by using a warm blooded animal like a bird
- place a small random integer just before the stack return address
    - if the value is overwritten/corrupted during the function call, there's a stack overflow attempt!
- use **-fno-stack-protector** flag in gcc/g++ to disable canaries


### checksec command

- program to check security settings on an ELF
- GDB/PEDA provides checksec command
- to run from bash, need to install checksec

In [36]:
! echo kali | sudo -S apt install checksec

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
checksec is already the newest version (2.6.0-2).
The following packages were automatically installed and are no longer required:
  catfish dh-elpa-helper docutils-common gir1.2-xfconf-0 libcfitsio9 libgdal31
  libmpdec3 libnginx-mod-http-geoip libnginx-mod-http-image-filter
  libnginx-mod-http-xslt-filter libnginx-mod-mail libnginx-mod-stream
  libnginx-mod-stream-geoip libpoppler123 libprotobuf23 libpython3.10
  libpython3.10-dev libpython3.10-minimal libpython3.10-stdlib libtiff5
  libzxingcore1 nginx-common nginx-core python-pastedeploy-tpl
  python3-alabaster python3-commonmark python3-docutils python3-imagesize
  python3-roman python3-snowballstemmer python3-speaklater python3-sphinx
  python3.10 python3.10-dev python3.10-minimal ruby3.0 ruby3.0-dev ruby3.0-doc
  sphinx-common
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [37]:
! checksec --file=./aslr.exe

RELRO           STACK CANARY      NX            PIE             RPATH      RUNPATH	Symbols		FORTIFY	Fortified	Fortifiable	FILE
Partial RELRO   No canary found   NX enabled    PIE enabled     No RPATH   No RUNPATH   50 Symbols	  No	0		1		./aslr.exe


### High level explanation

- https://blog.siphos.be/2011/07/high-level-explanation-on-some-binary-executable-security/

#### RELRO - Relocation Read Only

- the following resource provides good overview and example of RELRO
- https://www.redhat.com/en/blog/hardening-elf-binaries-using-relocation-read-only-relro
- Global Offset Table (GOT) and Procedure Linkage Table (PLT) can be made Read only

#### FORTIFY

- use compiler functionality to fortify certain variables that could potentially be overflown
- depends on the intelligence/feature of the compiler
